## Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## DEAP dataset

# Modifications:

## New ensemble approach (Reward Penalty based Weighted Ensemble [RPWE])


## Date: 7 Oct 2021 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import SimpleRNN
from keras.utils import np_utils
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


# from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
# from skmultiflow.meta import AdaptiveRandomForest

from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
def create_model(x):
    learning_rate = 0.05
    sgd = tf.keras.optimizers.SGD(lr=learning_rate)
    dim = x.shape[1]
    model = Sequential()
    model.add(Input(shape=(dim)))
    model.add(Dense(math.ceil((2/3)*dim),input_dim=dim,activation='sigmoid'))
    model.add(Dense(1,activation='sigmoid'))
    print(model.summary())
    model.compile(optimizer=sgd,
                loss='mse',
                metrics=['accuracy'])
    return model

In [7]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================

classifier = 'FFNN'
segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

init_m = 0
indx = 0
c=0
ccc =0


num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers

val_MSE = []
aro_MSE = []


# optimizer= 'SGD' #optimizer
# classifier = 'RNN_RPWE'+str(optimizer)
participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]


#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

all_weights_aro = []
all_weights_val = []


all_history_val = []
all_history_aro = []


mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()


print('Working with -->', classifier)
#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant): #Main loop participant loop

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = 'DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = 'DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # Resp data read from files
        ##===================================================
        def resp_data(p):
            file_resp = 'DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
            print(file_resp)
            resp_sig = pd.read_csv(file_resp,sep=',', header = None, engine='python')
            return resp_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_resp = resp_data(p)

        
        for jj in range(0,videos): #Video loop for each participants
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            resp_sig = grand_resp.loc[grand_resp.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            resp_val_prdt=[]
            resp_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                resp = np.array([resp_features]) #RESP BELT raw feature vector
                x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For Resp Belt data Model
                    #==============================
                    resp_model_val = create_model(x_resp)

                    resp_model_aro = create_model(x_resp)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    tmp_eeg_val = 0
                    tmp_eeg_aro = 0
                    
                    tmp_eda_val = 0
                    tmp_eda_aro = 0
                    
                    tmp_resp_val = 0
                    tmp_resp_aro = 0
                    
                    
                    y_pred_val_eeg = [2]
                    y_pred_aro_eeg = [2]
                    
                    y_pred_val_eda = [2]
                    y_pred_aro_eda = [2]
                    
                    y_pred_val_resp = [2]
                    y_pred_aro_resp = [2]
                    
                    #Fit the model once
                    hist_eeg_val = eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_eeg_aro = eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    hist_eda_val = eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_eda_aro = eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    hist_resp_val = resp_model_val.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_resp_aro = resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    c=c+1
                    
                else: 
                    
                    tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))[0][0][0]
                    tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))[0][0][0]
                    
                    tmp_eda_val = eda_model_val.predict(np.array([x_eda]))[0][0][0]
                    tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))[0][0][0]
                    
                    tmp_resp_val = resp_model_val.predict(np.array([x_resp]))[0][0][0]
                    tmp_resp_aro = resp_model_aro.predict(np.array([x_resp]))[0][0][0]
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                    y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                    y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]
                    
                    y_pred_val_resp = [1 if tmp_resp_val > 0.5 else 0]
                    y_pred_aro_resp = [1 if tmp_resp_aro > 0.5 else 0]
                    
                    #Fit the model once
                    hist_eeg_val = eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_eeg_aro = eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    hist_eda_val = eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_eda_aro = eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    hist_resp_val = resp_model_val.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    hist_resp_aro = resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification Resp Belt

                resp_acc_val = resp_acc_val.update(y_act_val[0], y_pred_val_resp[0])  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val[0], y_pred_val_resp[0]) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                

                #Arousal classification Resp Belt

                resp_acc_aro = resp_acc_aro.update(y_act_aro[0], y_pred_aro_resp[0])  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro[0], y_pred_aro_resp[0]) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            resp_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_resp_val))
            resp_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_resp_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, resp_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, resp_aro_MSE] #Arousal errors
            

            
            #=================================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #=================================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
                
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
            
            #============================================
            # Controls for the first time
            #============================================
            if ccc ==0:
                val_label = [0, 0, 0]
                aro_label = [0, 0, 0] 
                ccc =ccc+1
            else:
                val_label = [y_pred_val_eeg[0], y_pred_val_eda[0], y_pred_val_resp[0]]
                aro_label = [y_pred_aro_eeg[0], y_pred_aro_eda[0], y_pred_aro_resp[0]] 
            
            
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp[0])
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp[0])            
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            
#             #------------------------------------------
#             # Valence Class ensemble
#             #------------------------------------------
            
#             p_val = np.dot(val_label, w_val)
#             mer_val = 1 if p_val > 0.5 else 0
            
#             #------------------------------------------
#             # Arousal Class ensemble
#             #------------------------------------------            
#             p_aro = np.dot(aro_label,w_aro)
#             mer_aro = 1 if p_aro > 0.5 else 0
            
            # print([val_label, aro_label])
            # print([p_val,p_aro])
            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling
            
            #============================================
            # Weight update for Arousal
            #============================================
            
            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            mer_val = val_label[np.argmax(w_val)] #Final Class prediction Valence
            mer_aro = aro_label[np.argmax(w_aro)] #Final Class Prediction Valence
            
            all_weights_val.append(w_val)
            all_weights_aro.append(w_aro)
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                        eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                        y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                        eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                        y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_resp[0], y_act_aro[0], y_pred_aro_resp[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                        mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                        y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            
            all_history_val.append(['EEG',hist_eeg_val.history['loss'][0] ,'EDA',
                                    hist_eda_val.history['loss'][0] ,'RESP',hist_resp_val.history['loss'][0]])
            all_history_aro.append(['EEG',hist_eeg_aro.history['loss'][0] ,'EDA',
                                    hist_eda_aro.history['loss'][0] ,'RESP',hist_resp_aro.history['loss'][0]])
            
            print('MER model')
            

#             print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
#             print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
#             print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
#             print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
#             print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
#             print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
#             print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
#             print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')



Working with --> FFNN
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:1
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 768)               885504    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 769       
Total params: 886,273
Trainable params: 886,273
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_____________________________________________

MER model
===============================xxxx================================
Person: 1 Video:2
MER model
===============================xxxx================================
Person: 1 Video:3
MER model
===============================xxxx================================
Person: 1 Video:4
MER model
===============================xxxx================================
Person: 1 Video:5
MER model
===============================xxxx================================
Person: 1 Video:6
MER model
===============================xxxx================================
Person: 1 Video:7
MER model
===============================xxxx================================
Person: 1 Video:8
MER model
===============================xxxx================================
Person: 1 Video:9
MER model
===============================xxxx================================
Person: 1 Video:10
MER model
===============================xxxx================================
Person: 1 Video:11
MER model
============================

MER model
===============================xxxx================================
Person: 1 Video:27
MER model
===============================xxxx================================
Person: 1 Video:28
MER model
===============================xxxx================================
Person: 1 Video:29
MER model
===============================xxxx================================
Person: 1 Video:30


KeyboardInterrupt: 

In [29]:
all_history_aro = pd.DataFrame(all_history_aro)
all_history_aro.to_csv('all_error_aro.csv')

In [30]:
all_history_val = pd.DataFrame(all_history_val)
all_history_val.to_csv('all_error_val.csv')

In [16]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    571      1
      1      0    708
EEG aro CM:
             0      1
      0    539      4
      1      0    737
EDA Val CM:
             0      1
      0    415    157
      1    124    584
EDA aro CM:
             0      1
      0    367    176
      1    105    632
RESP Val CM:
             0      1
      0    413    159
      1    127    581
RESP Aro CM:
             0      1
      0    366    177
      1    104    633
ReMECS Val CM:
             0      1
      0    571      1
      1      1    707
ReMECS aro CM:
             0      1
      0    539      4
      1      1    736


In [13]:
all_weights_aro = pd.DataFrame(all_weights_aro)
all_weights_val = pd.DataFrame(all_weights_val)
all_weights_aro.to_csv('all_weights_aro.csv')
all_weights_val.to_csv('all_weights_val.csv')

,0,1,2
0,0.333333,3.333333e-01,3.333333e-01
1,0.333333,3.333333e-01,3.333333e-01
2,0.333333,3.333333e-01,3.333333e-01
3,0.333333,3.333333e-01,3.333333e-01
4,0.357076,3.253074e-01,3.176164e-01
...,...,...,...
1275,1.000000,3.554622e-16,4.034279e-16
1276,1.000000,3.554144e-16,4.031681e-16
1277,1.000000,3.554182e-16,4.031725e-16
1278,1.000000,3.554182e-16,4.031725e-16


In [15]:
#======================================
# Result Save For each Classifier
#======================================
classifier = 'RPWE'
filename = "DEAP/Final_Results/"
fname_eeg = filename + 'all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'all_person_RB'+'_' +classifier+'_results.csv'
fname_mer = filename + 'all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)